In [7]:
# !pip install requests

In [18]:
import requests
from datetime import datetime
import json
import os

In [15]:
# Sid: Establish API end point
server_endpoint = 'https://r5l50m40ne.execute-api.us-east-1.amazonaws.com/prod/robotics_s3_presignedurl'

filename = "README.md" 

# Sid: Logic to update the local image filename and insert a timestamp to ensure unique name in s3
file_name = os.path.basename(filename)
name, extension = os.path.splitext(file_name)
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
new_file_name = f"{name}_{timestamp}{extension}"


In [19]:
# Sid: Prepare the data payload as a dictionary
data = {
        'filename': new_file_name
}

# Sid: Establish the body ofthe request as the file and key/value of the new filename
data = {"body": json.dumps({"filename": new_file_name})}

# Sid: Convert the Python dictionary to a JSON string
data_json = json.dumps(data)

# Sid: Set the appropriate HTTP headers for sending JSON
headers = {'Content-Type': 'application/json'}

In [20]:
# Sid: Send a POST request to the API Gateway endpoint
response = requests.post(server_endpoint, data=data_json, headers=headers)

In [21]:
response

<Response [200]>

In [22]:
# Sid: Check if the request was successful
if response.status_code == 200:
        
    # Sid: Parse the response JSON to get the presigned URL
    presigned_url_response = response.json()
    presigned_url_body = json.loads(presigned_url_response['body'])
    presigned_url = presigned_url_body['url']

    # Sid: Confirm pre-signed url is in the response payload
    if presigned_url:
            
        # Sid: Open local file and prepare to upload a file to S3
        with open(filename, 'rb') as file_data:
                upload_response = requests.put(presigned_url, data=file_data)
                
                # Sid: Confirm success / failure of file upload into s3
                if upload_response.status_code == 200:
                    print("File was uploaded successfully.")
                else:
                    print(f"Failed to upload file: {upload_response.text}")
        
    # Sid: Return error message if pre-signed url not in response 
    else:
        print('Presigned URL not found in the response.')
    
# Sid: Return error message if pre-signed url not in response 
else:
    print('Failed to obtain presigned URL:', response.text)

File was uploaded successfully.
